In [1]:
!pip install beautifulsoup4 selenium pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.7/467.7 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.2 MB/s eta 0:00:00


In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from bs4 import BeautifulSoup
import pandas as pd

In [5]:
import time
import logging
import warnings
import argparse
import bs4 as bs
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from tqdm import tqdm

warnings.filterwarnings("ignore")
logging.basicConfig(level=logging.INFO)


def scrap(url, N_SCROLL, output_path, to_csv=True):
    options = webdriver.ChromeOptions()
    # options.add_argument('--headless')
    options.add_argument("--lang=id")
    driver = webdriver.Chrome(options=options)
    logging.info("opening url..")
    driver.get(url)

    # terbaru
    logging.info("click terbaru..")
    wait = WebDriverWait(driver, 10)
    menu = wait.until(lambda driver: driver.find_element( 'xpath',
        '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[7]/div[2]/button'))
    menu.click()
    terbaru = wait.until(lambda driver: driver.find_element('xpath',
        '//*[@id="action-menu"]/div[2]'))
    terbaru.click()

    time.sleep(1)

    # scroll down
    logging.info("scrolling down..")
    scrollable_div = driver.find_element("css selector",
        '#QA0Szd > div > div > div.w6VYqd > div.bJzME.tTVLSc > div > div.e07Vkf.kA9KIf > div > div > div.m6QErb.DxyBCb.kA9KIf.dS8AEf'
    )

    for _ in tqdm(range(int(N_SCROLL))):

        driver.execute_script(
            'arguments[0].scrollTop = arguments[0].scrollHeight',
            scrollable_div
        )
        time.sleep(1)

    # Scraping

    logging.info("get html..")
    resp = bs.BeautifulSoup(driver.page_source, 'lxml')
    reviews = resp.select(
        "#QA0Szd > div > div > div.w6VYqd > div.bJzME.tTVLSc > div > div.e07Vkf.kA9KIf > div > div > div.m6QErb.DxyBCb.kA9KIf.dS8AEf > div:nth-child(9)")
    soup_reviews = bs.BeautifulSoup(str(reviews), 'lxml')
    review = soup_reviews.find_all('div', class_='jftiEf fontBodyMedium')

    # Nama
    logging.info("get nama..")
    names = []
    for i in range(len(review)):
        names.append(review[i]['aria-label'])

    # Bintang
    logging.info("get bintang..")
    soup_bapak_bintang = bs.BeautifulSoup(str(review), 'lxml')
    bapak_bintang = soup_bapak_bintang.find_all('div', class_='DU9Pgb')

    bintangs = []
    for i in range(len(bapak_bintang)):
        soup_bintang = bs.BeautifulSoup(str(bapak_bintang[i]), 'lxml')
        bintang = soup_bintang.find('span', class_='kvMYJc')
        bintangs.append(int(bintang['aria-label'].split()[0]))

    # Komentar
    logging.info("get komentar..")
    komens = []
    for i in range(len(review)):
        try:
            komen = bs.BeautifulSoup(str(review[i]), 'lxml').find('div', class_="MyEned").text
        except:
            komen = ""
        komens.append(komen)

    data = pd.DataFrame({
        "nama": names,
        "bintang": bintangs,
        "komentar": komens
    })

    logging.info("to csv..")
    if to_csv:
        data.to_csv(output_path, index=False)
    else:
        return data